### 언어 모델 설정하기

In [2]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.01)
model.invoke("안녕하세요")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e665f7564b', 'id': 'chatcmpl-CBZJBkbwyqbtePhL797nI8fuZiFhx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1de623bf-8e6f-4528-8e50-f9a4879ff896-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 상태 설정하기

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    State 클래스는 TypedDict를 상속받습니다.

    속성:
        messages (Annotated[list[str], add_messages]): 메시지들은 "list" 타입을 가집니다.
        주석에 있는 'add_messages' 함수는 이 상태 키가 어떻게 업데이트되어야 하는지를 정의합니다.
        (이 경우, 메시지를 덮어쓰는 대신 리스트에 추가합니다.)
    """
    messages: Annotated[list[str], add_messages]

# StateGraph 클래스를 사용하여 State 타입의 그래프 생성
graph_builder = StateGraph(State)

### 도구 등록하기

In [4]:
from langchain_core.tools import tool
from datetime import datetime
import pytz
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

import bs4
from langchain_community.document_loaders import WebBaseLoader

# 도구 함수 정의
@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수"""
    try:
        tz = pytz.timezone(timezone)
        now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        result = f'{timezone} ({location}) 현재 시각 {now}'

        return result
    except pytz.UnknownTimeZoneError:
        return f"알 수 없는 타임존: {timezone}"

@tool
def get_web_search(query: str, search_period: str='m')-> str:
    """
    웹 검색을 수행하는 함수
    
    Args:
        query (str): 검색어
        search_perio (str): 검색 기간(e.g., "w" for past week (default), "m" for past month, "y" for past year, "d" for past day)
    
    Returns:
        str: 검색 결과
    """
    wrapper = DuckDuckGoSearchAPIWrapper(
        time=search_period
    )

    print('\n-------WEB SEARCH-------')
    print(query)
    print(search_period)

    search = DuckDuckGoSearchResults(
        api_wrapper=wrapper,
        result_separator=";\n"
    )

    searched = search.invoke(query)

    for i, result in enumerate(searched.split(';\n')):
        print(f'{i+1}. {result}')
    
    return searched

# 도구 바인딩
tools = [get_current_time, get_web_search]

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 현재 시각을 얻는 도구 get_current_time 실행 테스트하기

In [5]:
tools[0].invoke({"timezone": "Asia/Seoul", "location": "서울"})

'Asia/Seoul (서울) 현재 시각 2025-09-03 13:43:44'

### 웹 검색 도구 get_web_search 실행 테스트하기

In [7]:
tools[1].invoke({"query": "파이썬", "search_period": "m"})


-------WEB SEARCH-------
파이썬
m
1. snippet: 4 hours ago - Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly ..., title: Wikipedia Python (programming language) - Wikipedia, link: https://en.wikipedia.org/wiki/Python_(programming_language), snippet: 2 weeks ago - ไพทอน (อังกฤษ: Python) เป็นอินเทอร์พรีเตอร์ภาษาระดับสูงซึ่งสร้างโดยคีโด ฟัน โรสซึม โดยเริ่มใน พ.ศ. 2533 การออกแบบ..., title: Wikipedia ภาษาไพทอน - วิกิพีเดีย, link: https://th.wikipedia.org/wiki/ภาษาไพทอน, snippet: 2 weeks ago - 파이썬 설치부터 실행까지 완벽 가이드프로그래밍 초보자도 쉽게 따라할 수 있는 파이썬 설치 가이드서론: 파이썬 , 왜 배워야 할까요?최근 IT 분야에 관심이 많으시거나, 혹은 새롭게 프로그래밍을 시작해보고 싶은 분이라면 ..., title: Tistory 누구나 쉽게! 파이썬 설치부터 실행까지 완벽 가이드, link: https://informant-c.tistory.com/entry/누구나-쉽게-파이썬-설치부터-실행까지-완벽-가이드, snippet: 19시간 전 — 파이썬 의

'snippet: 4 hours ago - Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly ..., title: Wikipedia Python (programming language) - Wikipedia, link: https://en.wikipedia.org/wiki/Python_(programming_language), snippet: 2 weeks ago - ไพทอน (อังกฤษ: Python) เป็นอินเทอร์พรีเตอร์ภาษาระดับสูงซึ่งสร้างโดยคีโด ฟัน โรสซึม โดยเริ่มใน พ.ศ. 2533 การออกแบบ..., title: Wikipedia ภาษาไพทอน - วิกิพีเดีย, link: https://th.wikipedia.org/wiki/ภาษาไพทอน, snippet: 2 weeks ago - 파이썬 설치부터 실행까지 완벽 가이드프로그래밍 초보자도 쉽게 따라할 수 있는 파이썬 설치 가이드서론: 파이썬 , 왜 배워야 할까요?최근 IT 분야에 관심이 많으시거나, 혹은 새롭게 프로그래밍을 시작해보고 싶은 분이라면 ..., title: Tistory 누구나 쉽게! 파이썬 설치부터 실행까지 완벽 가이드, link: https://informant-c.tistory.com/entry/누구나-쉽게-파이썬-설치부터-실행까지-완벽-가이드, snippet: 19시간 전 — 파이썬 의 asyncio 는 I/O(입출력) 작업이 많은 환경에서 대기

### 도구 목록 확인하기

In [8]:
for tool in tools:
    print(tool.name, tool)

get_current_time name='get_current_time' description='현재 시각을 반환하는 함수' args_schema=<class 'langchain_core.utils.pydantic.get_current_time'> func=<function get_current_time at 0x0000026065F909A0>
get_web_search name='get_web_search' description='웹 검색을 수행하는 함수\n\nArgs:\n    query (str): 검색어\n    search_perio (str): 검색 기간(e.g., "w" for past week (default), "m" for past month, "y" for past year, "d" for past day)\n\nReturns:\n    str: 검색 결과' args_schema=<class 'langchain_core.utils.pydantic.get_web_search'> func=<function get_web_search at 0x0000026065F90900>
